In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
import featurewiz

Imported featurewiz: advanced feature engg and selection library. Version=0.0.42
output = featurewiz(dataname, target, corr_limit=0.70,
                    verbose=2, sep=',', header=0, test_data='',
                    feature_engg='', category_encoders='')
Create new features via 'feature_engg' flag : ['interactions','groupby','target']
                                


In [2]:
import xgboost as XGB

In [ ]:
df_clean = pd.read_csv('/Users/williamjames/Desktop/CSCI/487/final_data.csv')
df_clean = df_clean.drop(columns=['Unnamed: 0'])
df_home = df_clean.loc[(df_clean['location']==2)]
X = df_home.drop(columns=['win','location'])
y = df_home['win']



In [ ]:
pip install hyperopt

In [9]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }
def objective(space):
    clf= XGB.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))

    evaluation = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)


    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [14]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.5686136559697805, 'gamma': 8.816855788558225, 'max_depth': 6.0, 'min_child_weight': 6.0, 'reg_alpha': 43.0, 'reg_lambda': 0.5712339867238239}


In [5]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
params = {'colsample_bytree': 0.5686136559697805, 'gamma': 8.816855788558225, 'max_depth': 6, 'min_child_weight': 6, 'reg_alpha': 43, 'reg_lambda': 0.5712339867238239}

k_features = []
direction = []
features = []
score = []
accuracy = []
for k in range (5,20):
    gbm = XGB.XGBClassifier(**params,eval_metric='accuracy',verbosity=0)
    sfs1 = sfs(gbm, k_features=k, forward=True, verbose=0, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    accuracyCV = cross_val_score(clf, X[feat_names], y, scoring='accuracy', cv = 3)
    k_features.append(k)
    direction.append('forwards')
    features.append(feat_names)
    score.append(np.mean(accuracyCV))
forward = list(zip(k_features,direction,features,score))

In [ ]:
k_features = []
direction = []
features = []
score = []
accuracy = []
for k in range (5,20):
    gbm = XGB.XGBClassifier(**params,eval_metric='accuracy',verbosity=0)
    sfs1 = sfs(gbm, k_features=k, forward=False, verbose=0, scoring='neg_mean_squared_error')
    sfs1 = sfs1.fit(X, y)
    feat_names = list(sfs1.k_feature_names_)
    accuracyCV = cross_val_score(clf, X[feat_names], y, scoring='accuracy', cv = 3)
    k_features.append(k)
    direction.append('backwards')
    features.append(feat_names)
    score.append(np.mean(accuracyCV))
backwards = list(zip(k_features,direction,features,score))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
k_features = []
direction = []
scores = []
feats = []
accuracy = []
for i in range(5,19):
    rfe = RFE(estimator=XGB.XGBClassifier(**params,eval_metric='accuracy',verbosity=0), n_features_to_select=i)
    rfe.fit(X,y)
    clf = XGB.XGBClassifier(**params,eval_metric='accuracy',verbosity=0)
    support = rfe.support_
    feature_names = X_train.columns
    rfe_feats = feature_names[support]

    n_scores = cross_val_score(clf, X[rfe_feats], y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(i)
    direction.append('rfe')
    scores.append(np.mean(n_scores))
    feats.append(rfe_feats)

rfe_res = list(zip(k_features,direction,feats,scores))

In [ ]:
from sklearn.decomposition import PCA

k_features = []
direction = []
scores = []
feats = []

fk_features = []
method = []
scores = []
feats = []

for i in range(1,20):
    pca = PCA(n_components=i)
    X_pca = pca.fit_transform(X)

    # Get the transformed dataset
    X_pca = pd.DataFrame(X_pca)
    clf = XGB.XGBClassifier(**params,eval_metric='accuracy',verbosity=0)
    n_scores = cross_val_score(clf, X_pca, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
    k_features.append(i)
    method.append('pca')
    scores.append(np.mean(n_scores))
    feats.append('all')
pca_data = list(zip(k_features,direction,feats,scores))

In [18]:
bwd = pd.DataFrame(data=backwards[:],columns=['k_features','direction','features','score'])
fwd = pd.DataFrame(data=forward[:],columns=['k_features','direction','features','score'])
rfe_results = pd.DataFrame(data=rfe_res[:],columns=['k_features','direction','features','score'])
pca_results = pd.DataFrame(data=pca_data[:],columns=['k_features','direction','features','score'])
results = pd.concat([bwd,fwd,rfe_results,pca_results])

In [19]:
results.sort_values('score',ascending=False)

,k_features,direction,features,score
13,18,rfe,"Index(['HHI_assists', 'G_PTS', 'F_PTS', 'C_PTS...",0.674001
1,6,rfe,"Index(['TO_rate', 'OREB_rate', 'opp_EFG', 'opp...",0.673103
7,12,rfe,"Index(['TO_rate', 'EFG', 'FT_rate', 'OREB_rate...",0.671756
13,18,forwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.668612
2,7,rfe,"Index(['TO_rate', 'FT_rate', 'OREB_rate', 'opp...",0.667265
...,...,...,...,...
4,9,backwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.637180
2,7,backwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.634486
3,8,backwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.630445
1,6,backwards,"[HHI, HHI_assists, G_PTS, F_PTS, C_PTS, starte...",0.628199


In [20]:
results.to_csv('XGBoost_Results.csv',index=False)

In [6]:
from sklearn.model_selection import cross_val_score
params = {'colsample_bytree': 0.5686136559697805, 'gamma': 8.816855788558225, 'max_depth': 6, 'min_child_weight': 6, 'reg_alpha': 43, 'reg_lambda': 0.5712339867238239}

fts = []
direction = []
k_features = []
accuracy = []
clf = XGB.XGBClassifier(**params,eval_metric='accuracy',verbosity=0)
feats = ['opp_play_percent', 'play_percent', 'TO_rate', 'OREB_rate', 'FT_rate', 'opp_TO_rate', 'opp_OREB_rate', 'starter_MIN', 'C_PTS', 'F_PTS', 'opp_FT_rate']
X = df_home[feats]
y = df_home['win']
n_scores = cross_val_score(clf, X, y, scoring='accuracy', cv=3, n_jobs=-1, error_score='raise')
fts.append(feats)
k_features.append(len(feats))
direction.append('featurewiz')
accuracy.append(np.mean(n_scores))
fwiz = list(zip(k_features,direction,fts,accuracy))
fw_res = pd.DataFrame(data=fwiz[:],columns=['k_features','direction','features','score'])


0.6560395150426583


In [17]:
results = pd.read_csv('XGBoost_Results.csv')
results = results.loc[(results['direction']!='pca')]
pca_results = pd.DataFrame(data=pca_data[:],columns=['k_features','direction','features','score'])
results = pd.concat([results,fw_res,pca_results])

In [19]:
results.sort_values('score',ascending=False)

,k_features,direction,features,score
43,18,rfe,"Index(['HHI_assists', 'G_PTS', 'F_PTS', 'C_PTS...",0.674001
31,6,rfe,"Index(['TO_rate', 'OREB_rate', 'opp_EFG', 'opp...",0.673103
37,12,rfe,"Index(['TO_rate', 'EFG', 'FT_rate', 'OREB_rate...",0.671756
28,18,forwards,"['HHI', 'HHI_assists', 'G_PTS', 'F_PTS', 'C_PT...",0.668612
32,7,rfe,"Index(['TO_rate', 'FT_rate', 'OREB_rate', 'opp...",0.667265
...,...,...,...,...
0,1,pca,all,0.637629
4,9,backwards,"['HHI', 'HHI_assists', 'G_PTS', 'F_PTS', 'C_PT...",0.637180
2,7,backwards,"['HHI', 'HHI_assists', 'G_PTS', 'F_PTS', 'C_PT...",0.634486
3,8,backwards,"['HHI', 'HHI_assists', 'G_PTS', 'F_PTS', 'C_PT...",0.630445


In [20]:
results.to_csv('XGBoost_Results.csv',index=False)